# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
import numpy as np
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import FunctionTransformer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report

import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

import ssl

In [2]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /Users/emreakyuz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/emreakyuz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emreakyuz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine.connect())

In [4]:
df.drop('original', axis=1, inplace=True)

In [5]:
X =  df['message']         
Y =  df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def mytokenizer(text):


     """
    normalize, tokenize, remove stop words and lemmatize text
    
    Parameters:
    text: Text to be processed

    Returns:
    clean_tokens: Processed text
    """

    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
                  
    # Tokenize text

    tokens = word_tokenize(text)

    # Remove stop words

    tokens = [w for w in tokens if w not in stopwords.words("english")]

    # Lemmatization

    lemmatizer = WordNetLemmatizer()

    clean_tokens = []

    for tok in tokens:

        clean_tok = lemmatizer.lemmatize(tok).lower().strip()

        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=mytokenizer)),
    ('clf',MultiOutputClassifier(lgb.LGBMClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function mytokenizer at 0x15069d300>)),
                ('clf', MultiOutputClassifier(estimator=LGBMClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
pred = pipeline.predict(X_test)

In [10]:
f1_score(y_test, pred, average='micro')

0.6162696714686444

In [11]:
print(classification_report(y_test, pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               request       0.79      0.56      0.66       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.78      0.69      0.73      2164
          medical_help       0.63      0.31      0.42       435
      medical_products       0.64      0.28      0.39       279
     search_and_rescue       0.61      0.21      0.31       136
              security       0.29      0.02      0.04        96
              military       0.61      0.32      0.42       158
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.68      0.70       335
                  food       0.82      0.81      0.82       584
               shelter       0.74      0.61      0.67       468
              clothing       0.73      0.47      0.57        70
                 money       0.56      0.29      0.38       112
        missing_people       0.67      

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
# second grid search for clf

parameters = {

    'clf__estimator__learning_rate': [0.1,0.5]}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose = 3)

In [13]:
cv.fit(X_train, y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV 1/2] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=25;, score=0.425 total time=  42.4s
[CV 2/2] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=25;, score=0.420 total time=  41.8s
[CV 1/2] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50;, score=0.428 total time=  48.3s
[CV 2/2] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50;, score=0.425 total time=  48.3s
[CV 1/2] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100;, score=0.418 total time= 1.1min
[CV 2/2] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100;, score=0.415 total time= 1.0min
[CV 1/2] END clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=25;, score=0.408 total time=  41.9s
[CV 2/2] END clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=25;, score=0.405 total time=  42.1s
[CV 1/2] END clf__estimator__learn

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function mytokenizer at 0x15069d300>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=LGBMClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [0.1, 0.3, 0.5],
                         'clf__estimator__n_estimators': [25, 50, 100]},
             verbose=3)

In [14]:
cv.best_params_

{'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=mytokenizer)),
    ('clf',MultiOutputClassifier(lgb.LGBMClassifier(learning_rate =  0.1, n_estimators =  50)))
])

In [21]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function mytokenizer at 0x15069d300>)),
                ('clf',
                 MultiOutputClassifier(estimator=LGBMClassifier(n_estimators=50)))])

In [22]:
pred = pipeline.predict(X_test)

In [23]:
f1_score(y_test, pred, average='micro')

0.609868928296068

In [19]:
print(classification_report(y_test, pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               request       0.83      0.47      0.60       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.78      0.56      0.65      2164
          medical_help       0.69      0.19      0.30       435
      medical_products       0.75      0.25      0.38       279
     search_and_rescue       0.56      0.17      0.26       136
              security       0.00      0.00      0.00        96
              military       0.57      0.18      0.28       158
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.66      0.71       335
                  food       0.83      0.80      0.82       584
               shelter       0.79      0.55      0.65       468
              clothing       0.76      0.49      0.59        70
                 money       0.62      0.23      0.34       112
        missing_people       0.77      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
my_pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=mytokenizer,ngram_range=(1, 2))),
    ('clf',MultiOutputClassifier(xgb.XGBClassifier()))
])

my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function mytokenizer at 0x15069d300>)),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric=None,
                                                               feature_type...
                                                               gpu_id=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...)))])

In [25]:
predictions = my_pipeline.predict(X_test)

In [26]:
f1_score(y_test, predictions, average='micro')

0.5998473428108004

### 9. Export your model as a pickle file

In [29]:
file = 'ml_model.pkl'
pickle.dump(pipeline, open(file, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.